FProfile.jl provides an alternative interface around Julia's [sampling profiler](https://docs.julialang.org/en/latest/manual/profile/). Please read the introduction of that document before proceeding if you've never used `@profile`.

# Profiling

You can build a profile by calling `@fprofile(code, delay=0.001, n_samples=1000000)`:

In [2]:
using FProfile, Calculus

pd = @fprofile second_derivative(sin, 1.0)

INFO: Recompiling stale cache file /Users/cedric/.julia/lib/v0.6/FProfile.ji for module FProfile.


ProfileData(45 backtraces)

`@fprofile(N, ...)` is a shorthand for `@fprofile(for _ in 1:N ... end)`:

In [3]:
pd = @fprofile 1000000 second_derivative(sin, 1.0)

ProfileData(1104 backtraces)

`ProfileData` merely wraps the internal data in `Base.Profile`. Do not forget that Julia compiles code the first time a function is run; if you do not want to measure compilation time, execute your code once before profiling.

# Flat view

FProfile's `flat` report is a [dataframe](http://juliadata.github.io/DataFrames.jl/stable/man/getting_started/#Getting-Started-1), however no particular knowledge of dataframes is necessary. I'll provide a few common operations below.

In [4]:
using DataFrames

df = flat(pd)
head(df, 5)   # show only the first 5 rows (the 5 rows with the highest count)

,count_percent,stackframe,specialization,method,file,function,module
1,92.93,"execute_request(::ZMQ.Socket, ::IJulia.Msg) at execute_request.jl:154","MethodInstance for execute_request(::ZMQ.Socket, ::IJulia.Msg)","execute_request(socket, msg) in IJulia at /Users/cedric/.julia/v0.6/IJulia/src/execute_request.jl:115",/Users/cedric/.julia/v0.6/IJulia/src/execute_request.jl,IJulia.execute_request,IJulia
2,92.93,anonymous at <missing>:?,MethodInstance for missing_info(),missing_info() in FProfile at /Users/cedric/.julia/v0.6/FProfile/src/FProfile.jl:271,/Users/cedric/.julia/v0.6/FProfile/src/FProfile.jl,FProfile.missing_info,FProfile
3,92.93,(::IJulia.##14#17)() at task.jl:335,MethodInstance for (::IJulia.##14#17)(),(::IJulia.##14#17)() in IJulia at task.jl:335,task.jl,IJulia.#14,IJulia
4,92.93,eventloop(::ZMQ.Socket) at eventloop.jl:8,MethodInstance for eventloop(::ZMQ.Socket),eventloop(socket) in IJulia at /Users/cedric/.julia/v0.6/IJulia/src/eventloop.jl:2,/Users/cedric/.julia/v0.6/IJulia/src/eventloop.jl,IJulia.eventloop,IJulia
5,92.93,macro expansion at FProfile.jl:40 [inlined],MethodInstance for missing_info(),missing_info() in FProfile at /Users/cedric/.julia/v0.6/FProfile/src/FProfile.jl:271,/Users/cedric/.julia/v0.6/FProfile/src/FProfile.jl,FProfile.missing_info,FProfile


The first column shows what fraction of backtraces (in %) go through the `method at file:line_number` in the `stackframe` column. It's the same quantity as in `Base.Profile.print()`, except for recursive calls: if `f(1)` calls `f(0)`, that's 2 counts in Base's report, but only 1 count in FProfile.

The other columns unpack the `stackframe`; they are useful for selecting subsets of the table. For instance, if I only care about the `derivative` function, I might use

```julia
    df[df[:function].===derivative, :]
```

It is common to focus optimization efforts on one module at a time (... the one you're developing). `flat(pd, MyModule)` filters out other modules and adds a useful column: `end_count_percent` measures how much `MyModule`-specific work is done on that line.

For instance, in the code below, while the `do_computation()` call takes a long time (it has a high `count_percent`), it merely calls another `Main` function, so it has a low `end_count_percent`. `sum_of_sin` has `end_count_percent = ~87%` because while it calls `sum` and `sin`, those are defined in another module (`Base`), and counted as external.

Why 87% and not 100%? That's a [bug in the profiler](https://github.com/JuliaLang/julia/issues/9689). 

`flat(pd, (Module1, Module2, ...))` is also accepted.

In [5]:
@noinline do_computation(n) = sum_of_sin(n)
@noinline sum_of_sin(n) = sum(sin, 1:n)
pd2 = @fprofile do_computation(10000000)
flat(pd2, Main)

,count_percent,end_count_percent,stackframe,specialization,method,file,function,module
1,87.43,0.0,do_computation(::Int64) at In[5]:1,MethodInstance for do_computation(::Int64),do_computation(n) in Main at In[5]:1,In[5],do_computation,Main
2,87.43,87.43,sum_of_sin(::Int64) at In[5]:2,MethodInstance for sum_of_sin(::Int64),sum_of_sin(n) in Main at In[5]:2,In[5],sum_of_sin,Main


It pays to make sure that functions with a high `end_count_percent` are [well optimized](https://docs.julialang.org/en/latest/manual/performance-tips/).

Another way to reduce the level of detail is to aggregate by `:specialization, :method, :file, :function`, or `:module`.

In [6]:
flat(pd, combineby=:method)

,count_percent,method,file,function,module
1,92.93,"include_string(txt::String, fname::String) in Base at loading.jl:515",loading.jl,include_string,Base
2,92.93,eventloop(socket) in IJulia at /Users/cedric/.julia/v0.6/IJulia/src/eventloop.jl:2,/Users/cedric/.julia/v0.6/IJulia/src/eventloop.jl,IJulia.eventloop,IJulia
3,92.93,(::IJulia.##14#17)() in IJulia at task.jl:335,task.jl,IJulia.#14,IJulia
4,92.93,"execute_request(socket, msg) in IJulia at /Users/cedric/.julia/v0.6/IJulia/src/execute_request.jl:115",/Users/cedric/.julia/v0.6/IJulia/src/execute_request.jl,IJulia.execute_request,IJulia
5,92.93,"include_string(mod::Module, code::String, fname::String) in Compat at /Users/cedric/.julia/v0.6/Compat/src/Compat.jl:464",/Users/cedric/.julia/v0.6/Compat/src/Compat.jl,include_string,Compat
6,92.93,missing_info() in FProfile at /Users/cedric/.julia/v0.6/FProfile/src/FProfile.jl:271,/Users/cedric/.julia/v0.6/FProfile/src/FProfile.jl,FProfile.missing_info,FProfile
7,92.66,"second_derivative(f::Function, x::Number) in Calculus at /Users/cedric/.julia/v0.6/Calculus/src/derivative.jl:71",/Users/cedric/.julia/v0.6/Calculus/src/derivative.jl,Calculus.second_derivative,Calculus
8,43.03,"derivative(f::Function, ftype::Symbol, dtype::Symbol) in Calculus at /Users/cedric/.julia/v0.6/Calculus/src/derivative.jl:2",/Users/cedric/.julia/v0.6/Calculus/src/derivative.jl,Calculus.derivative,Calculus
9,16.58,"finite_difference_hessian(f::Function, g::Function, x::Number, dtype::Symbol) in Calculus at /Users/cedric/.julia/v0.6/Calculus/src/finite_difference.jl:224",/Users/cedric/.julia/v0.6/Calculus/src/finite_difference.jl,Calculus.finite_difference_hessian,Calculus
10,12.95,"finite_difference(f::Function, x::T, dtype::Symbol) where T<:Number in Calculus at /Users/cedric/.julia/v0.6/Calculus/src/finite_difference.jl:48",/Users/cedric/.julia/v0.6/Calculus/src/finite_difference.jl,Calculus.finite_difference,Calculus


Other useful dataframe commands:

```julia
df[[:count_percent, :method]]   # select only those two columns
sort(df, cols=:end_count_percent, rev=true)  # sort by end_count_percent
showall(df)   # show the whole dataframe
```

See also `?flat` for more options.

# Tree view

FProfile's tree view looks the same as `Base.Profile.print(format=:tree)`:

In [36]:
tree(pd)

 1026 ./task.jl:335; (::IJulia.##14#17)()
  1026 ...ulia/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
   1026 ...c/execute_request.jl:154; execute_request(::ZMQ.Socket, :...
    1026 ...Compat/src/Compat.jl:464; include_string(::Module, ::Str...
     1026 ./loading.jl:515; include_string(::String, ::String)
      1026 ./<missing>:?; anonymous
       1026 ...ile/src/FProfile.jl:40; macro expansion
        1026 ./profile.jl:23; macro expansion
         1024 ...le/src/FProfile.jl:55; macro expansion
          2    .../src/derivative.jl:0; second_derivative(::Functio...
          1021 .../src/derivative.jl:71; second_derivative(::Functio...
           9   ...src/derivative.jl:0; derivative(::Function, ::Sy...
           1   ...src/derivative.jl:2; derivative(::Function, ::Sy...
           465 ...src/derivative.jl:3; derivative(::Function, ::Sy...
           5   ...ite_difference.jl:0; finite_difference_hessian(:...
           178 ...ite_difference.jl:224; finite_difference_hessian(:...
    

However, the return value is an indexable and filterable datastructure:

In [8]:
tr_calc = filter(sf->get_module(sf)===Calculus, tr)

 3    ...lus/src/derivative.jl:0; second_derivative(::Function, ::F...
 1023 ...lus/src/derivative.jl:71; second_derivative(::Function, ::F...
  11  ...lus/src/derivative.jl:0; derivative(::Function, ::Symbol,...
  2   ...lus/src/derivative.jl:2; derivative(::Function, ::Symbol,...
  467 ...lus/src/derivative.jl:3; derivative(::Function, ::Symbol,...
  6   .../finite_difference.jl:0; finite_difference_hessian(::Func...
  179 .../finite_difference.jl:224; finite_difference_hessian(::Func...
   4   .../finite_difference.jl:0; finite_difference(::Calculus.##1...
   27  .../finite_difference.jl:27; finite_difference(::Calculus.##1...
   114 .../finite_difference.jl:55; finite_difference(::Calculus.##1...
    3  ./math.jl:300; finite_difference(::Base.#sin, :...
    2  .../finite_difference.jl:0; finite_difference(::Base.#sin, :...
    28 .../finite_difference.jl:27; finite_difference(::Base.#sin, :...
    6  .../finite_difference.jl:54; finite_difference(::Base.#sin, :...
    67 .../finite

In [9]:
tr_calc[2, 4]

6 ...rc/finite_difference.jl:0; finite_difference_hessian(::Function...


The functions `get_specialization, get_method, get_file, get_function, get_module, is_C_call, is_inlined` all accept a `StackFrame` argument, and are useful to select the calls you care about (note that C calls are excluded by default; see `?tree`).

Use `prune(tree, n_levels)` to limit the level of details, and see `?filter_bloodline` to keep more context around the filtered nodes.

# Backtraces

(if you want to build your own analysis)

The raw Profile data is available either through `Base.Profile.retrieve()`, or through `pd.data, pd.lidict`. However, you might find `FProfile.backtraces(::ProfileData)` more immediately useful. `tree(pd::ProfileData)` is defined as `tree(backtraces(pd))`, and similarly for `flat`, so you can modify the backtraces and get a tree/flat view of the results.

In [10]:
count, trace = backtraces(pd)[1]  # get the first unique backtrace
@show count                       # the number of times that trace occurs in the raw data
trace

count = 2


11-element Array{StackFrame,1}:
 (::IJulia.##14#17)() at task.jl:335                                  
 eventloop(::ZMQ.Socket) at eventloop.jl:8                            
 execute_request(::ZMQ.Socket, ::IJulia.Msg) at execute_request.jl:154
 include_string(::Module, ::String, ::String) at Compat.jl:464        
 include_string(::String, ::String) at loading.jl:515                 
 anonymous at <missing>:?                                             
 macro expansion at FProfile.jl:40 [inlined]                          
 macro expansion at profile.jl:23 [inlined]                           
 macro expansion at FProfile.jl:55 [inlined]                          
 second_derivative(::Function, ::Float64) at derivative.jl:71         
 derivative(::Function, ::Symbol, ::Symbol) at derivative.jl:3        

Use the `get_method, get_file, ...` functions on `StackFrame` objects (see above).